# TensorFlow Deploy - Introduction 

TensorFlow Deploy (briefly: `tfd`) is a storage, versioning and model management service created in [TensorFlow](www.tensorflow.org) (`tf`) and compliant with [TensorFlow Serving](www.tensorflow.org/serving) (`tfs`). Its main objective is to simplify the process of a model going into production environment to the maximum - a data scientist is to focus on producing high quality tested models instead of devops. Additionally, `tfd` strongly integrates with `tfs` and allows to utilize even the most advanced functionalities. Another key fact is that not only models, but also `tf` modules compliant with [TensorFlow Hub](www.tensorflow.org/hub) (`tfh`) can be stored in the service.

In [ ]:
import tensorflow as tf
import tensorflow_deploy_utils as tfd

In [ ]:
tf.__version__, tfd.__version__

## Model preparation

It's time for a few typical notes useful to each data science guy. So what we do is - we download the data, prepare it, then we design the model, teach it, and when the results are satisfactory, we save it.

### Data preparation

In [ ]:
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_data = train_data / 255.0
test_data = test_data / 255.0
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

### Model preparation

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=5, input_shape=(28, 28)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

### Teaching, evaluating and saving the model 

In [ ]:
history = model.fit(x=train_data, y=train_labels, validation_split=0.2, epochs=1, batch_size=64)

In [ ]:
loss, accuracy = model.evaluate(x=test_data, y=test_labels, batch_size=64, verbose=0)
print("Test accuracy: {a}".format(a=accuracy))

This basic model using sample data gave us accuracy of 87-89%. That will do for the first run ;)

In [ ]:
EXPORT_PATH = "/tmp/models/mnist/1"
tf.saved_model.save(model, EXPORT_PATH)

Our model is now saved locally on the hard drive and we are almost ready to implement it into production. Just one more step - we need a model description. We assume that each model that is implemented into production (no matter the label) has to be described. Basing on our experience, we realize that coming back to one model after some time give us no clue on how it differs from its previous versions. That is why TensorFlow Deploy does not accept a model without its README.md file, even though `tfs` does not really require it. `Tfd` verifies the contents of the package while uploading in order to make sure that it gets both the model together with the readme file. 

## Project IDs

**Attention!** This is the most important part of the notebook. It is crucial to understand and use it in order to let `tfd` work properly. Basing on these IDs, `tfd` will know which `tfs` instance to inform about the new model (or module). Additionally, sticking to this naming convention will allow you to keep your model repository neat and tidy. Following versions of the same model are also versioned, described and labeled. This will allow you to find a desired model, even after a long time, while informing you about the differences between versions at the same time. 

If you are a bigger company with many data scientists grouped into different teams working at multiple projects simultaneously, you will also appreciate `tfd` and its naming convention. It will give you the chance to check easily how the project develops and which version is currently in production. You can also run A/B tests thanks to labeled models.  

Let's explain what particular IDs mean:
- `team` - name of the team where you work, develop the project and the model 
- `project` - name of the project where you create models
- `name` - name of the model you create
- `label` - (optional) label for the model which will let us enquire `tfs` for it


A model (`team`, `project`, `name`) can have many different versions. Each of the version can have different labels. However, if we keep uploading or implementing a model with the same label, then the label will apply to the most recent version only. For the remaining models, which will become unlabelled, there will only be a possibility of asking `tfs` about them by their version numbers - unless we assign them different labels. 

**Attention!** We assume that when the label is not assigned, all models go into production labelled as `canary` by default. A production model is considered to be labelled as `stable` and restricted. We cannot implement a model labelled as `stable` right away. We have to implement it first with another label and then use `set_stable` method to set the label for the model. This has to prevent non-working or non-tested models by accidentally going into production. We recommend creating a 'tfs' client so that it always asks for a model labelled `stable` and, for example, sends 1% queries to the model labelled as `canary`.

**Attention!** We assume that each ID can consist of lower case letters and digits. If we use upper case letters, these will be transformed into lower case signs. Signs that are not allowed will result in an error and lack of communication with `tfd`. Maximum lenght of an ID is 32 signs.

That is all we need to know in order to use `tfd` smoothly. Let's go back to the core topic of the notebook and implement our model. 

In [ ]:
TEAM = "demo"
PROJECT = "intro"
NAME = "mnist"
LABEL = "basic"  # optional

In [ ]:
tfd_cursor = tfd.TFD(team=TEAM, project=PROJECT, name=NAME, label=LABEL, host="tfd")

Now we have the `tfd` cursor, which allows us to run operations using the service, manage models or modules. While creating the cursor, we also check whether `tfd` exists in the desired location (`http://tfd:9500` by deafult) - we will get an error and the cursor will not be created if it is not there. As mentioned previously, the error will also result in the ID being misnamed (too long ID or unallowed signs).

## Model description

In [ ]:
DESCRIPTION = """
This is a smiple model version solving MNIST issue. A simple two-layer neuron network 
with a few neurons has been applied to solve it.
"""

tfd_cursor.generate_model_readme(dst_path=EXPORT_PATH, description=DESCRIPTION, 
                                 metrics={"accuracy": accuracy, "loss": loss})

We have got a README with a model descriprion as well as results (metrics) within our model. Such a set can be implemented into production. 

## Production Deploy

In [ ]:
response = tfd_cursor.deploy_model(EXPORT_PATH)
print(response)

**GREAT SUCCESS!** Our first model has just been implemented into production, which means we have such a `tfs` instance that has its model assigned and is ready for external queries.

## Production check

In case of doubts whether the model really is available in production - i.e. whether there is a `tfs` instance serving it.

In [ ]:
from helpers import grpc_client, show, show_label

In [ ]:
predictions = grpc_client(dataset=test_data, host=f"tfs-{TEAM}-{PROJECT}", port=8500, 
                          model_name=NAME, model_label=LABEL)

This is what a single response from `tfs` with a mnist model looks like.

In [ ]:
predictions[0]

We use a `show_label` function to return a model decision in a concise form.

In [ ]:
show_label(array=predictions[0].outputs["dense_1"].float_val)

Let's see what is the model's reponse.

In [ ]:
show(array=test_data[0], 
     title=f"Label true: {show_label(test_labels[0])} predicted: {show_label(predictions[0].outputs['dense_1'].float_val)}")

# Summary 

Everything works exactly as designed.

Thanks to `tfs` the way from idea to production has become short and easy. At the same time, the naming conception of `tfs` models and instances creates order both in a repository of a junior data science as well as a big corporation running tens of projects and hundreds of models. It is also worth noticing that the whole implementation process is not disturbing any regular tasks of a data scientist and does not require a pipeline construction.

You are very welcome to read our other notebooks in order to explore more advanced **TensorFlow Deploy** functions.